<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import sys
import os
from pathlib import Path
import networkx as nx
from numba import jit
from dask.distributed import Client
pd.options.mode.chained_assignment = None  # default='warn'

client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/122/1,Cluster Workers: 1 Cores: 2 Memory: 13.65 GB




---


**Tentative code for rnc2 temporal network**

@time

Conditions -> Boolean mask -> Contacts -> Edgelist


---



In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/Academia/01_Publications/00_Alex/005_RNC2')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

**Conditions**


---



In [0]:
def wind_scenario(wind_data):  
    i = np.random.randint(0, wind_data.values.shape[0])
    w = wind_data.values[i, 2]
    dist = wind_data.values[i, 1]
    b = wind_data.values[i, 3]
    bear_max = b + 45  # wind direction
    bear_min = b - 45
    if b == 360:
        bear_max = 45
    if b <= 0:  # should not be necessary
        bear_min = 0
    if b == 999:
        bear_max = 999
        bear_min = 0
    return bear_max, bear_min, dist # wind characteristics, bearing and distance

# ignition ?
def condition0(t, e, c):
    rng = np.random.uniform(0, 1, size=e.values.shape[0])
    return rng < e.IgnProb_bl.values

# is this edge.target a new edge.source ?
def condition1(t, e, c):
    previous_target = e.target.values[c[:, t-1] == True]
    return np.in1d(e.target.values, previous_target)

# wind direction
def condition2(t, e, c, w_bear_max, w_bear_min):
    return (e.bearing.values < w_bear_max) & (e.bearing.values > w_bear_min)

# wind speed
def condition3(t, e, c, w_dist):
    return e.distance.values < w_dist

# not burnt in previous fires ?
def condition4(t, e, c):
      mask = c[:,:-1].sum(axis=1)
      mask = mask == 0 # == 0 means that no fire previously occured
      return mask

**Main**


---



In [0]:
for scenario in range(1):
    time = 0 
    while True in contact_matrix[:,-1]:  # while still some fires burning
        print(" ******* scenario : {},  time : {} *******".format(scenario, time))
        if time == 0:
            list_of_activation = [] # to record fires
            contact_matrix = np.full((edgelist.shape[0],1), True) # keep track of contacts in time
            wind_bearing_max, wind_bearing_min, wind_distance = wind_scenario(wind_data) # wind conditions for each scenarios
            c0 = condition0(time, edgelist, contact_matrix)
            contact_matrix[:, time] = c0 # initial ignitions
            print("condition 0 == True : {} ignitions".format(c0[c0 == True].shape))
        else:
            c1 = condition1(time,edgelist,contact_matrix)
            c2 = condition2(time,edgelist,contact_matrix,wind_bearing_max, wind_bearing_min)
            c3 = condition3(time,edgelist,contact_matrix,wind_distance)
            c4 = condition4(time,edgelist,contact_matrix)
            print("condition 1 == True : {} target -> source".format(c1[c1 == True].shape))
            print("condition 2 == True : {} propagation because of wind direction".format(c2[c2 == True].shape))
            print("condition 3 == True : {} propagation because of wind speed (distance)".format(c3[c3 == True].shape))
            print("condition 4 == True : {} not burnt already".format(c4[c4 == True].shape))

            propagation = c1 & c2 & c3 & c4
            print("propagation == True : {} propagations".format(propagation[propagation == True].shape))
            contact_matrix = np.c_[contact_matrix, propagation]
            print("fire burning ? {}".format(True in contact_matrix[:, -1]))
        
        # active edges at time
        active_edges = edgelist[contact_matrix[:, time] == True]
        active_edges['scenario'] = scenario
        active_edges['time'] = time
        # record acrive edges to list
        list_of_activation.append(active_edges.values)
        time += 1
    else:
        # # record active edges in dask and export to parquet
        da_scenario = da.concatenate(list_of_activation, axis=1)
        # generate dask dataframe
        col = ['source', 'target', 'distance', 'bearing', 'IgnProb_bl', 'scenario', 'time']
        ddf = dd.from_dask_array(da_scenario, columns=col)
        # save to parquet
        ddf.to_parquet(folder / 'output' / 'scenario_{}.parquet'.format(scenario))